In [4]:
import os
# import pandas library as pd
import pandas as pd
from bs4 import BeautifulSoup
from lxml import etree
from os import listdir
from os.path import isfile, join

# create an Empty DataFrame object
df = pd.DataFrame()
print(df)

#this function is being used in extract data funtion 
#to get columns per row and assigning them 
#if missing it will assign empty string 
def getTextFromTag(columns, index):
    columnText = ""
    try:
        columnText = columns[index].get_text()
    except:
        pass
    return columnText

#this is extract data function
#this function is callled in reports per day function to extract data 
#it reads one file and adds to data frame
#it takes data frame and returns data frame updated per files 
def extractdata(df, filename):

    with open(filename, 'r') as f:

        contents = f.read()
        #this is lxml method to read
        #soup = BeautifulSoup(contents, 'lxml')

        #this is beautiful soup html parser 
        soup = BeautifulSoup(contents, "html.parser")
        dom = etree.HTML(str(soup))

        print("File: " + soup.title.text)
        if type(soup.title.text) == None:
            return df
            print ("this is not complete df")
            
        
        try:
            linerSize = dom.xpath('//*[@id="AutoNumber1"]/thead/tr[2]/td/table/tbody/tr[2]/td[2]/table/tbody/tr[2]/td[1]')[0].text
            date = dom.xpath('//*[@id="AutoNumber1"]/thead/tr[1]/td/table/tbody/tr[1]/td[2]/table/tbody/tr[3]/td')[0].text
            wellno = dom.xpath('//*[@id="AutoNumber1"]/thead/tr[1]/td/table/tbody/tr[1]/td[3]/table[1]/tbody/tr[2]/td')[0].text
            rigname = dom.xpath('//*[@id="AutoNumber1"]/thead/tr[1]/td/table/tbody/tr[1]/td[4]/table/tbody/tr[2]/td')[0].text
        except:
            linerSize = ""
            date = ""
            wellno = ""
            rigname = ""
        
        if linerSize is None:
            linerSize = "None"
        #assign soup items in to table and rows rows is table body and table rows 
        table = soup.find("table", {"id": "AutoNumber2"})     
        #printing the table
        #print(table)
                
        rows = table.find("tbody").find_all("tr")

        for row in rows:
            
            #this will grab all the columns for each row 
            columns = row.find_all("td")

            #printing length of columns
            print(len(columns))

            Time_in_Hrs = getTextFromTag(columns, 0)
            OperationStartTime = getTextFromTag(columns, 1)
            OperationEndtTime = getTextFromTag(columns, 3)
            lateral = getTextFromTag(columns, 4)
            holesize = getTextFromTag(columns, 5)
            majoroperation = getTextFromTag(columns, 7)
            action = getTextFromTag(columns, 8)
            operation = getTextFromTag(columns, 9)
            company = getTextFromTag(columns, 10)
            holeDepthStart = getTextFromTag(columns, 11)
            holeDepthEnd = getTextFromTag(columns, 12)
            eventDepthStart = getTextFromTag(columns, 13)
            eventDepthEnd = getTextFromTag(columns, 14)
            unique_id = date + "-" +OperationStartTime + "-" + OperationEndtTime



            if len(columns) == 16:
                SummaryOfOperation = getTextFromTag(columns, 15)
            else: 
                SummaryOfOperation = getTextFromTag(columns, 17)
            #checking for lost time
            if company == "LT Summary" or holeDepthStart == "LT Summary":
                #returning df after checking for lost time
                return df
            if operation is None or company is None:
                continue    
            #checking for extra line of information about the depth 
            #and company this will ignore that line. it is un necessary 
            if company == "Depth":
                
                #it will skip the line
                continue
            data = {
                "unique_id":unique_id,
                "date": date,
                "operation_start_time": OperationStartTime,
                "operation_end_time": OperationEndtTime,
                "time_in_hrs": Time_in_Hrs,
                "rig_name": rigname,
                "well_no": wellno,
                "service_company_name": company, 
                "liner_size": linerSize,
                "hole_size":holesize,
                "lateral":lateral,
                "operation": operation,
                "major_operation": majoroperation,
                "action": action,
                "hole_start_depth" : holeDepthStart,
                "hole_end_depth" : holeDepthEnd,
                "event_start_depth" : eventDepthStart,
                "event_end_depth" : eventDepthEnd,
                "summary_of_operation" : SummaryOfOperation       
            }
            
            df = df.append(data, ignore_index = True)

    return df

#this takes in df and morning reports folder
#it goes through all files in the folder 
#returns data frame 
def readReportsPerDay(df, morningReportFolder):

    mypath = morningReportFolder + "/reports"
    #mypath += "/reports"
    onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    
    count = 1

    for filename in onlyfiles:

        filename = mypath + "/" + filename

        print(str(count) + " " + filename)

        df = extractdata(df, filename)
        count += 1

    return df


#this funtion will call the reports per day function 
#this is to be used for running the program in folder
#it goes thru the all morning report folders per day inside specific folder 
def runInRunningFolder(df):

    for morningReportFolder in listdir(os.getcwd()):

        if ".ipynb" in morningReportFolder or ".csv" in morningReportFolder or ".DS_Store" in morningReportFolder or ".xlsx" in morningReportFolder or ".json" in morningReportFolder or ".pbix" in morningReportFolder or ".zip" in morningReportFolder:
            continue

        print(morningReportFolder + "\n")

        df = readReportsPerDay(df, morningReportFolder)

    return df


print("Extract function is done, please continue.")



#to display to make sure we have empty data frame     
#display(df)      
print(df)



print("Setting up libraries, and empty data frame is done, please continue.\n")
print("-Yemane")


Empty DataFrame
Columns: []
Index: []
Extract function is done, please continue.
Empty DataFrame
Columns: []
Index: []
Setting up libraries, and empty data frame is done, please continue.

-Yemane


In [5]:
#this is where we run the code for each item in directory 
#this will list curent working directory and rung th code 

print(os.getcwd())


df = runInRunningFolder(df)

print(df)

/Volumes/YEMANE 256G/2022/2022/6-28 feb
MR 2-06-2022

1 MR 2-06-2022/reports/088TE QTIF-279.html
File: 088TE
16
16
16
16
16
16
16
16
16
16
16
16
16
2 MR 2-06-2022/reports/095TE MNIF-225.html
File: 095TE
16
3 MR 2-06-2022/reports/910TE DMMM-524.html
File: 910TE
16
16
16
16
16
16
16
16
4 MR 2-06-2022/reports/ADC-12 ABQQ-199.html
File: ADC-12
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
5 MR 2-06-2022/reports/ADC-14 HRDH-694.html
File: ADC-14
16
16
16
16
16
16
16
16
6 MR 2-06-2022/reports/ADC-15 HRDH-1915.html
File: ADC-15
16
16
16
7 MR 2-06-2022/reports/ADC-16 JAHM-6.html
File: ADC-16
16
16
16
16
16
16
16
16
16
16
16
16
8 MR 2-06-2022/reports/ADC-17 ZULF-1210.html
File: ADC-17
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
9 MR 2-06-2022/reports/ADC-21 MHWZ-125104.html
File: ADC-21
16
16
16
16
16
16
16
16
16
16
16
16
10 MR 2-06-2022/reports/ADC-23 RMLH-15.html
File: ADC-23
16
16
16
11 MR 2-06-2022/reports/ADC-29 ABMK-4.html


In [6]:
print(df)

                  unique_id        date operation_start_time  \
0      02/06/2022-0500-0600  02/06/2022                 0500   
1      02/06/2022-0600-0800  02/06/2022                 0600   
2      02/06/2022-0800-0900  02/06/2022                 0800   
3      02/06/2022-0900-1030  02/06/2022                 0900   
4      02/06/2022-1030-1500  02/06/2022                 1030   
...                     ...         ...                  ...   
46865  02/28/2022-1815-2200  02/28/2022                 1815   
46866  02/28/2022-2200-2330  02/28/2022                 2200   
46867  02/28/2022-2330-0100  02/28/2022                 2330   
46868  02/28/2022-0100-0200  02/28/2022                 0100   
46869  02/28/2022-0200-0500  02/28/2022                 0200   

      operation_end_time time_in_hrs rig_name   well_no service_company_name  \
0                   0600        01.0    088TE  QTIF-279                  BHT   
1                   0800        02.0    088TE  QTIF-279                

Thsis is to fix endcoing in data


In [7]:
df.to_excel("feb-6-28-not-25.xlsx")

In [8]:
df_updated = df.drop(['hole_size','event_start_depth','event_end_depth'],axis = 1)

In [9]:
df_updated

,unique_id,date,operation_start_time,operation_end_time,time_in_hrs,rig_name,well_no,service_company_name,liner_size,lateral,operation,major_operation,action,hole_start_depth,hole_end_depth,summary_of_operation
0,02/06/2022-0500-0600,02/06/2022,0500,0600,01.0,088TE,QTIF-279,BHT,7,0,BIT,CT,RW,19400,19400,"RIH 6 1/8"" CLEANOUT ASSY ON 4"" DP & HAD SLACK-..."
1,02/06/2022-0600-0800,02/06/2022,0600,0800,02.0,088TE,QTIF-279,RIG,7,0,MPMP,CT,CIRC,19400,19400,CIRC 2 CYCLE EACH HEADED BY 30 BBL HI-VIS PILL...
2,02/06/2022-0800-0900,02/06/2022,0800,0900,01.0,088TE,QTIF-279,RIG,7,0,STDP,CT,TI,19400,19400,"RIH 6 1/8"" CLEANOUT ASSY ON 4"" DP FREE ON ELEV..."
3,02/06/2022-0900-1030,02/06/2022,0900,1030,01.5,088TE,QTIF-279,RIG,7,0,MPMP,CT,CIRC,19400,19400,"CIRC 1 BTM-UP WITH 350 GPM, FLOW CHECK, STATIC..."
4,02/06/2022-1030-1500,02/06/2022,1030,1500,04.5,088TE,QTIF-279,RIG,7,0,STDP,CT,TO,19400,19400,"POOH WITH 6 1/8"" CLEANOUT ASSY ON 4"" DP TO TOP..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46865,02/28/2022-1815-2200,02/28/2022,1815,2200,03.8,ZPEC-2,HZEM-836,HCM,None,0,CSG,CSG,CMC,2610,2610,"HPJSM ,SWITCH LINE TO CMT UNIT FLASH & P/T SUR..."
46866,02/28/2022-2200-2330,02/28/2022,2200,2330,01.5,ZPEC-2,HZEM-836,HCM,None,0,CSG,CSG,TOP,2610,2610,HPJSM. PERFORM 1ST TOP JOB:HALLIBURTON MIX & P...
46867,02/28/2022-2330-0100,02/28/2022,2330,0100,01.5,ZPEC-2,HZEM-836,RIG,None,0,CSG,CSG,WOC,2610,2610,WOC.
46868,02/28/2022-0100-0200,02/28/2022,0100,0200,01.0,ZPEC-2,HZEM-836,HCM,None,0,CSG,CSG,TOP,2610,2610,HPJSM. PERFORM 2ND TOP JOB:HALLIBURTON MIX & P...


In [10]:
df_updated.to_csv("dropped_feb_6_28_22_not_25th.csv")

In [ ]:
for column in df.columns:
    df[column] = df[column].strng.replace(r'\W',"")

In [15]:
df.to_excel("1-jun-30-jun with out 21-jun.xlsx", encoding = ascii)

NameError: name 'utf' is not defined

In [3]:
df

,unique_id,date,operation_start_time,operation_end_time,time_in_hrs,rig_name,well_no,service_company_name,liner_size,hole_size,lateral,operation,major_operation,action,hole_start_depth,hole_end_depth,event_start_depth,event_end_depth,summary_of_operation
0,06/17/2022-0500-0600,06/17/2022,0500,0600,01.0,HI-2,BRRI-440,RIG,None,8 1/2,0,BHA,DRLG,HT,8491,8491,,,"CON'T P/U & M/U BHI 8-1/2"" RSS MWD/LWD ASSEMBL..."
1,06/17/2022-0600-0700,06/17/2022,0600,0700,01.0,HI-2,BRRI-440,RIG,None,8 1/2,0,NA,DRLG,OTH,8491,8491,,,PROGRAMMED MWD/LWD TOOLS.
2,06/17/2022-0700-0815,06/17/2022,0700,0815,01.2,HI-2,BRRI-440,RIG,None,8 1/2,0,BHA,DRLG,HT,8491,8491,,,"P/U FLOAT SUB , FILTER SUB AND CIRCULATION SUB..."
3,06/17/2022-0815-0830,06/17/2022,0815,0830,00.2,HI-2,BRRI-440,RIG,None,8 1/2,0,SPHDP,DRLG,TI,8491,8491,113,392,"RIH 8-1/2"" RSS MWD/LWD ASSEMBLY ON 5"" HWDP."
4,06/17/2022-0830-0900,06/17/2022,0830,0900,00.5,HI-2,BRRI-440,RIG,None,8 1/2,0,MPMP,DRLG,CIRC,8491,8491,,,SHALLOW TESTED DIRECTIONAL TOOLS - OK.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59758,06/18/2022-1300-1500,06/18/2022,1300,1500,02.0,EXL-IVA,SFNY-1650,RIG,None,,0,NA,RM,RM,0,0,,,HOLD BOW PRE-LOAD.
59759,06/18/2022-1500-1600,06/18/2022,1500,1600,01.0,EXL-IVA,SFNY-1650,RIG,None,,0,NA,RM,RM,0,0,,,DUMP BOW PRE-LOAD.
59760,06/18/2022-1600-1615,06/18/2022,1600,1615,00.2,EXL-IVA,SFNY-1650,RIG,None,,0,NA,RM,RM,0,0,,,JACK UP RIG 2' & LEVEL.
59761,06/18/2022-1615-0000,06/18/2022,1615,0000,07.8,EXL-IVA,SFNY-1650,RIG,None,,0,NA,RM,RM,0,0,,,COMMENCE STBD PRE-LOAD AS PROCEDURE.


In [ ]:
#filename = my_Master_name 
filename = df.iloc[0][0]
filename = filename.replace("/", "-")
print(filename)
#print(my_master_name)

df.to_csv(filename + ".csv", sep=',', encoding='utf-8')
#df.to_csv(my_master_name + ".csv", sep=',', encoding='utf-8')

In [1]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'